Robust to outliers

If the difference between the predicted value and the real value is small, square it If it’s large, take its absolute value


1. Control Flow Ops

2. Comparison Ops

3. Logical Ops

4. Debugging Ops

Instead of doing inference with placeholders and feeding in data later, do inference directly with real Data 

In [3]:
import tensorflow_sample_program.CS20.utils as utils
import os
import tensorflow as tf

fileDir = os.path.dirname(os.path.realpath('__file__'))
filename = os.path.join(fileDir, '../Kaggle/tensorflow_sample_program/CS20/data/birth_life_2010.txt')
DATA_FILE = os.path.abspath(os.path.realpath(filename))
data, n_samples = utils.read_birth_life_data(DATA_FILE)



In [15]:
dataset = tf.data.Dataset.from_tensor_slices((data[:,0],data[:,1]))

print(dataset.output_types)
print(dataset.output_shapes)

(tf.float32, tf.float32)
(TensorShape([]), TensorShape([]))


Can also create Dataset from files

1. tf.data.TextLineDataset(filenames)
2. tf.data.FixedLengthRecordDataset(filenames)
3. tf.data.TFRecordDataset(filenames)

Create an iterator to iterate through samples in Dataset

In [16]:
# Iterate through the dataset exactly once. No need to initialization.
iterator = dataset.make_one_shot_iterator()

#Iterates through the dataset as many times as we want. Need to initialize with each epoch.
iterator_initialisable = dataset.make_initializable_iterator()


In [17]:
iterator = dataset.make_one_shot_iterator()
X, Y = iterator.get_next()

with tf.Session() as sess:
    print(sess.run([X,Y]))
    print(sess.run([X,Y]))

[1.822, 74.82825]
[3.869, 70.81949]


In [22]:
iterator = dataset.make_initializable_iterator()

sess = tf.Session()
for i in range(10):
    sess.run(iterator.initializer)
    total_loss = 0
    
sess.close()

1. dataset.shuffle(1000)
2. dataset.repeat(100)
3. dataset.batch(128)
4. dataset.map(lambda x : tf.one_hot(x,10))

1. For protyping, feed dict can be faster and easier to write(pythonic)
2. tf.data is tricky to use when you have complicated preprocessing or mutiple data sourcs
3. NLP data is normally just a sequence of integers. 
In this case, transferring the data over to GPU is pretty quick, 
so the speedup of tf.data isn't that large

How does Tensorflow know what varibales to update?


optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

_, l = sess.run([optimizer, loss], feed_dict={X: x, Y:y})


Session looks at all trainable variables that loss depends on and update them.

In [24]:
# Specify if a variable should be trained or not, By default, all variables are trainable.
tf.Variable(initial_value=5, trainable=True)

<tf.Variable 'Variable:0' shape=() dtype=int32_ref>

Advanced optimizers work better when tuned, but are generally harder to tune.

1. tf.train.GradientDescentOptimizer
2. tf.train.AdagradOptimizer
3. tf.train.MomentumOptimizer
4. tf.train.AdamOptimizer
5. tf.train.FtrlOptimizer
6. tf.train.RMSPropOptimizer

In [37]:
from tensorflow.examples.tutorials.mnist import input_data
MNIST = input_data.read_data_sets('data/mnist', one_hot=True)

print(type(MNIST))

# No immediate way to convert Python generators to tf.data.Dataset

Extracting data/mnist\train-images-idx3-ubyte.gz


Extracting data/mnist\train-labels-idx1-ubyte.gz
Extracting data/mnist\t10k-images-idx3-ubyte.gz
Extracting data/mnist\t10k-labels-idx1-ubyte.gz


Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x00000000150D2C50>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x00000000150DC358>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x00000000150DC400>)


In [1]:
import tensorflow_sample_program.CS20.utils as utils

mnist_folder = 'data/mnist/'
utils.download_mnist(mnist_folder)



#train_data = tf.data.Dataset.from_tensor_slices(train)


In [15]:
import os
import tensorflow_sample_program.CS20.utils as utils
import tensorflow as tf

fileDir = os.path.dirname(os.path.realpath('__file__'))
filename = os.path.join(fileDir, '../Kaggle/data/mnist/')
DATA_FILE = os.path.abspath(os.path.realpath(filename))
print(DATA_FILE)

train, val, test = utils.read_mnist(DATA_FILE, flatten=True)

train_data = tf.data.Dataset.from_tensor_slices(train)
#train_data = train_data.shuffle(10000)
test_data = tf.data.Dataset.from_tensor_slices(test)

# iterator = train_data.make_initializable_iterator()

iterator = tf.data.Iterator.from_structure(train_data.output_types,train_data.output_shapes)

img, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)
test_init = iterator.make_initializer(test_data)

with tf.Session() as sess:
    sess.run(train_init)
    print(sess.run([img, label]))
    print(sess.run([img, label]))


with tf.Session() as sess:
    sess.run(test_init)
    print(sess.run([img, label]))
    print(sess.run([img, label]))

C:\Users\m.kumar\manish\kaggle\Machine-Learning\Kaggle\Kaggle\data\mnist


[array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.    

[array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.    